https://github.com/sonyisme/keras-recommendation/blob/master/keras-master/examples/movieslensDSSM.py

In [2]:
from keras.models import Sequential
from keras.models import Model
from keras.layers import Activation, Dropout, Dense, Reshape
from keras.layers import Merge
import numpy as np
# fix random seed for reproducibility
np.random.seed(7)

Using TensorFlow backend.


### Test Model

In [3]:
user= np.zeros([1024,3883])
items= np.zeros([1024,2,6039])
y_train=np.zeros([1024,2])

In [13]:
user = np.array([[1,0,0],[1,0,0],[0,1,0],[1,1,0]])
y_train = np.array([1,1,1,0])
items = np.array(  [['i','am','at','school'],['i','am','an','apple'],['queen','and','king','ob'],['queen','is','his','mom']])

In [14]:
# 4 Users 3 Features
user.shape

(4, 3)

In [15]:
# 4 Items 4 Features
items.shape

(4, 4)

In [16]:
# 4 Labels
y_train.shape

(4,)

In [17]:
# create user model
userModel = Sequential()
#userModel.add(Dense(3,input_dim=3))
userModel.add(Dense(6, input_shape=(3,)))
userModel.add(Activation('tanh'))
userModel.add(Dropout(0.4))
# userModel.add(Dense(2, input_dim=3))
userModel.add(Dense(12, input_shape=(6,)))
userModel.add(Activation('tanh'))
# userModel.add(Reshape((2,1)))
userModel.add(Reshape((12,1)))

In [22]:
# create item model
itemModel = Sequential()
#itemModel.add(Dense(3,input_dim=3))
itemModel.add(Dense(6, input_shape=(4,)))
#itemModel.add(Conv1D(filters, kernel_size, strides=1, padding='valid', dilation_rate=1, \
#       activation=None, use_bias=True, kernel_initializer='glorot_uniform', \
#       bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, \
#       activity_regularizer=None, kernel_constraint=None, bias_constraint=None))
itemModel.add(Activation('tanh'))
itemModel.add(Dropout(0.4))
# itemModel.add(Dense(2,input_dim=3))
itemModel.add(Dense(12, input_shape=(6,)))
itemModel.add(Activation('tanh'))
# itemModel.add(Reshape((2,1)))
itemModel.add(Reshape((12,1)))

## Load Data

### User-Article Data

In [5]:
import pandas as pd
import numpy as np
import scipy as sp
import json
import types
import ast 
import re
import string

f_ngram=open('data/ngram.txt')
voca=f_ngram.read().split('\n')
voca_num=len(voca)
print voca_num

f_title=open('data/tr_article_title_ngram.txt')
title=f_title.readlines()
tr_num=len(title)
print tr_num

10945
168276


In [11]:
article_sample = np.array(50)
for i in range(50):
    code_sample = np.zeros(voca_num)
    cnt = title[i].split()
    for j in cnt:
        n = voca.index(j)
        code_sample[n] = 1
    if i == 0:
        article_sample = code_sample 
    else:
        article_sample = np.vstack([article_sample, code_sample])
items = article_sample

In [18]:
items.shape

(50, 10945)

### User Feature Sample Data

In [12]:
f_id = open('data/user_id_code.txt')
uid = f_id.readlines()

f_brand = open('data/user_brand_code.txt')
ubrd = f_brand.readlines()

In [13]:
#get the total num of uid
f_idset = open('data/user_id_set.txt')
totalid = len(f_idset.readlines())
print totalid

#get the total num of uid
f_brdset = open('data/user_brand_set.txt')
totalbrd = len(f_brdset.readlines())
print totalbrd

7857
50


In [16]:
uid_sample = np.array(50)
for i in range(50):
    ucode = np.zeros(totalid)
    ucode_brd = np.zeros(totalbrd)
    n = int(uid[i])
    ucode[n] = 1
    m = int(ubrd[i])
    ucode_brd[m] = 1
    this_code = np.concatenate([ucode, ucode_brd])
    if i == 0:
        uid_sample = this_code
    else:
        uid_sample = np.vstack([uid_sample, this_code])
user = uid_sample

In [19]:
user.shape

(50, 7907)

In [60]:
y_train = np.ones(25)
y_train = np.concatenate([y_train, np.zeros(25)])
y_train.shape

(50,)

## Training

In [61]:
# create user model
userModel = Sequential()
#userModel.add(Dense(3,input_dim=3))
userModel.add(Dense(300, input_shape=(7907,)))
userModel.add(Activation('tanh'))
userModel.add(Dropout(0.4))
# userModel.add(Dense(2, input_dim=3))
userModel.add(Dense(100, input_shape=(300,)))
userModel.add(Activation('tanh'))
# userModel.add(Reshape((2,1)))
userModel.add(Reshape((100,1)))

In [62]:
# create item model
itemModel = Sequential()
#itemModel.add(Dense(3,input_dim=3))
itemModel.add(Dense(300, input_shape=(10945,)))
itemModel.add(Activation('tanh'))
itemModel.add(Dropout(0.4))
# itemModel.add(Dense(2,input_dim=3))
itemModel.add(Dense(100, input_shape=(300,)))
itemModel.add(Activation('tanh'))
# itemModel.add(Reshape((2,1)))
itemModel.add(Reshape((100,1)))

In [19]:
model = Sequential()
model.add(Merge([userModel,itemModel], mode='cos', dot_axes=1, output_shape=(1,)))
model.add(Reshape((1,)))

In [20]:
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
# Fit the model
model.fit([user, items],y_train, nb_epoch = 50, batch_size=10) # epochs=150 in keras 2+

Epoch 1/50


ValueError: could not convert string to float: queen

In [66]:
# evaluate the model
scores = model.evaluate([user,items], y_train)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

32/50 [==================>...........] - ETA: 0s
acc: 56.00%


In [67]:
# calculate predictions
predictions = model.predict([user,items])
# round predictions
rounded = [round(x[0]) for x in predictions]
print(rounded)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, -0.0, -0.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]


In [68]:
import numpy as np
np.array(predictions).shape

(50, 1)